# Normal Auto_FEP Workflow

## 1. 配体预处理

1. 配体"_name" 属性修改
1. 检查化学有效性
1. 依据拓扑找出最大微扰map组
1. 找出最大charge 相同组
1. 重写valid mols 到 指定文件夹

参数说明：
- `path`: 当前ligands所在目录
- `path_valid_mols`: 筛选后有效分子目录
- `path_valid_info`: 有效分子信息
- `path_invalid_info`: 无效分子信息

**Step 1. 创建工作目录，后续所有步骤的输出将写到这个目录中**

In [1]:
!mkdir -p output

**Step 2. 生成参数配置文件**

In [2]:
%%writefile output/check_ligs_input.json
{
  "path": "docs/example/lig_data",
  "path_valid_mols": "output/lig_data_output",
  "path_valid_info": "output/valid_info.csv",
  "path_invalid_info": "output/invalid_info.csv"
}

Writing output/check_ligs_input.json


**Step 3. 对指定的小分子 做配体预处理**

In [3]:
!python check_mols.py -f output/check_ligs_input.json

base-name jmc_27.mol
base-name jmc_23.mol
base-name ejm_46.mol
base-name ejm_50.mol
base-name ejm_44.mol
valid_Chemical_Rule: [<rdkit.Chem.rdchem.Mol object at 0x7fd096aa5ac0>, <rdkit.Chem.rdchem.Mol object at 0x7fd096a0a200>, <rdkit.Chem.rdchem.Mol object at 0x7fd096a0a270>, <rdkit.Chem.rdchem.Mol object at 0x7fd096a0a2e0>, <rdkit.Chem.rdchem.Mol object at 0x7fd096a0a350>]
the loop is: 5
len_mol_list: (<rdkit.Chem.rdchem.Mol object at 0x7fd096aa5ac0>, <rdkit.Chem.rdchem.Mol object at 0x7fd096a0a200>, <rdkit.Chem.rdchem.Mol object at 0x7fd096a0a270>, <rdkit.Chem.rdchem.Mol object at 0x7fd096a0a2e0>, <rdkit.Chem.rdchem.Mol object at 0x7fd096a0a350>)
matches ((0, 1, 2, 3, 4, 5, 28, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 32, 16, 17, 18, 33, 31, 30, 29, 23, 27, 26),)
c1([H])c([H])c(Cl)c(C(=O)N(c2c([H])c([H])nc(N(C=O)[H])c2[H])[H])c(Cl)c1[H]
the loop is: 4


## 2. 生成微扰 map

edge 属性：
1. perturbed_atoms: 微扰原子数
1. desirablility: 边的可信度(默认为1)
1. atom_map: 分子的原子匹配关系（需要3D可视化）
1. cost: 计算该pair的代价；与微扰原子数相关
1. similarity: 分子相似性
1. MCS: pair 共有子结构

参数说明：
- `input`: ligands所在位置
- `progress_file`: 输出的 plk 文件
- `map_info_file`: 输出的 json 文件表示的微扰图结构
- `graph_file`: 输出的 svg 文件表示的微扰图结构
- `map_type`: 微扰map 类型
- `map_bias`: 微扰map 中心分子

**Step 1. 生成参数配置文件**

In [4]:
%%writefile output/gen_map.ini
[generate_perturbation_map]

input = output/lig_data_output
progress_file = output/progress.pkl
map_info_file = output/generdated_map_info.json
graph_file = output/best_graph.svg
map_type = star
map_bias = ejm_46

Writing output/gen_map.ini


**Step 2. 生成微扰图**

In [5]:
!python generate_perturbation_map_atom_map_wangwei.py --config_file output/gen_map.ini

the inputs are output/lig_data_output
the file is output/lig_data_output/jmc_27.mol
the file is output/lig_data_output/jmc_23.mol
the file is output/lig_data_output/ejm_46.mol
the file is output/lig_data_output/ejm_50.mol
the file is output/lig_data_output/ejm_44.mol
full_thermograph: DiGraph with 5 nodes and 4 edges
pos: {'jmc_27': array([9.99999986e-01, 2.18556937e-08]), 'jmc_23': array([-3.57647606e-08,  1.00000000e+00]), 'ejm_50': array([-9.9999997e-01, -6.5567081e-08]), 'ejm_44': array([ 1.98715071e-08, -9.99999956e-01]), 'ejm_46': [0.0, 0.0]}


## 3. 修改微扰map

功能：增加和删除指定的边，并删除度为0的节点

**Step 1. 生成参数文件**

In [16]:
%%writefile output/modify_map_input.json
{
  "add_edge": [
    [
      "jmc_23",
      "ejm_46"
    ]
  ],
  "del_edge": [
    [
      "ejm_46",
      "jmc_23"
    ]
  ]
}

Writing output/modify_map_input.json


**Step 2. 调用接口，根据参数修改微扰图**

In [ ]:
!python modify_perturbation_map_zcc.py -f output/modify_map_input.json

## 4. 修改指定边的AtomMap

**Step 1. 生成参数文件**

In [ ]:
%%writefile output/modify_atommap_input.json
{
  "add_atom_pair": {
    "FXR_76,FXR_84": [
      [
        0,
        3
      ]
    ],
  },
  "del_atom_pair": {
    "FXR_84,FXR_85": [
      [
        52,
        59
      ],
      [
        56,
        55
      ]
    ]
  }
}

**Step 2. 调用接口，根据参数修改原子图**

In [ ]:
!python modified_lig_atommap.py -f modify_atommap_input.json

## 5. align 配体到口袋

参数说明：
- `ref_mol_path`: 以经过docking 的分子作为参考分子
- `lig_data_path`: 待匹配的分子

**Step 1. 生成参数文件**

In [ ]:
%%writefile output/align_ligs_input.json
{
  "ref_mol_path": "output/lig_data/ejm_46.mol",
  "lig_data_path": "output/lig_data_output"
}

**Step 2. 调用接口，对齐配体到口袋**

In [ ]:
!python alignment_ligands_zcc.py -f output/align_ligs_input.json

## 6. 生成配体拓扑文件(.itp)

**Step 1. 使用sobtop生成拓扑文件（这里配体文件路径必须为绝对路径）**

In [ ]:
!python gen_lig_itp_sob.py -input /home/work/pyautofep/PyAutoFEP/aligned_lig_data

**Step 2. 使用acpype生成拓扑文件**

In [ ]:
!python gen_lig_itp_acp.py -input aligned_lig_data

## 7. 生成dual_topl文件

配置文件参数说明：
- `input_ligands`      : ligands 所在目录
- `structure`          : protein 所在目录
- `perturbations_dir`  : 生成文件所在目录
- `gmx_bin_local`      : gromacs 命令
- `buildsys_forcefield`: protein 采用力场 1: amber03
- `md_temperature`     : MD模拟温度:*k
- `md_length`          : MD模拟时间长度：5000 ps

**Step 1. 生成参数文件**

In [ ]:
%%writefile output/step2.ini
[prepare_dual_topology]

input_ligands = aligned_lig_data
structure = receptor_data/5q17_processed.pdb
perturbations_dir = tutorial
gmx_bin_local = gmx_mpi
buildsys_forcefield = 1
md_temperature = 298.15
md_length = 5000.0 

**Step 2. 调用接口，生成dual topl文件**

In [ ]:
!python prepare_dual_topology_zcc.py --config_file=step2.ini --output_hidden_temp_dir False